Company wants to know which of these candidates are really wants to work for the company after training or looking for a new employment because it helps to reduce the cost and time as well as the quality of training or planning the courses and categorization of candidates. Information related to demographics, education, experience are in hands from candidates signup and enrollment.

This dataset designed to understand the factors that lead a person to leave current job for HR researches too. By model(s) that uses the current credentials,demographics,experience data you will predict the probability of a candidate to look for a new job or will work for the company, as well as interpreting affected factors on employee decision.

The whole data divided to train and test . Target isn't included in test but the test target values data file is in hands for related tasks. A sample submission correspond to enrollee_id of test set provided too with columns : enrollee _id , target

from kaggle

In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns',None)
#from pandas import datetime
#from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline    
#plt.rcParams['font.sans-serif'] = ['SimHei'] 
#plt.rcParams['axes.unicode_minus'] = False 
#from sklearn.metrics import mean_squared_error as MSE
#from tabulate import tabulate


from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
import warnings
warnings.filterwarnings("ignore")#忽略警告提醒

In [2]:
df=pd.read_csv('/Users/Ruby/Desktop/train.csv')
df

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1.0
19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1.0
19155,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0.0
19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


In [3]:
df.info()#查看資料所有欄位

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

資料中有些沒有用的符號，像在欄位city中，數據都以city_xx表示，但是分析時並不需要前面「city_」這幾個字，因此先把這幾個字移除，再來工作經歷欄位中，有> ,<的符號也移除。

In [4]:
#移除資料中不需要用到的符號
def deal_usable_data(df):
    df['city'] = df['city'].map(lambda x : x.lstrip('city_'))
    df['experience'] = df['experience'].astype(str).map(lambda x : x.lstrip('< >'))
    df['last_new_job'] = df['last_new_job'].astype(str).map(lambda x : x.lstrip('< >'))
    return df


In [5]:
df=deal_usable_data(df)

In [6]:
#製作表格，查看遺失值佔的比例
def nan (df):
    a = df.isnull().sum()
    b = pd.DataFrame(a,columns = ['total NAN'])
    total = df.isnull().sum().sum()
    b['%'] = round((df.isnull().sum()/total),2)*100
    return b

In [7]:
nan(df)

,total NAN,%
enrollee_id,0,0.0
city,0,0.0
city_development_index,0,0.0
gender,4508,22.0
relevent_experience,0,0.0
enrolled_university,386,2.0
education_level,460,2.0
major_discipline,2813,14.0
experience,0,0.0
company_size,5938,29.0


In [8]:
#資料中，「enrollee_id」這個欄位代表受試者的id，此欄位對於預測沒有幫助，所以刪除
df = df.drop(['enrollee_id'],axis=1)

此份資料中有許多遺失值，且佔的比例很高，有些遺失值占比高達30％，因此決定利用隨機森林模型，利用完整的資料部分去預測遺失值，並將預測結果來用來填補遺失值。

In [9]:
from sklearn.ensemble import RandomForestClassifier
tree = RandomForestClassifier()

In [10]:
#tr_gender為無遺失值的部分，
#te_gender為只有遺失值的部分，要利用無遺失值的部分來預測遺失的部分，

tr_gender = df[df['gender'].notnull()].astype(str).apply(LabelEncoder().fit_transform)
te_gender =  df[df['gender'].isnull()].fillna(0).astype(str).apply(LabelEncoder().fit_transform)

#利用無遺失值的資料建立隨機森林模型
tr_y_gender = tr_gender['gender']
tr_x_gender = tr_gender.drop(['gender'],axis=1)
tree.fit(tr_x_gender,tr_y_gender)

#預測遺失值
te_x_gender = te_gender.drop( ['gender'],axis=1)
pred_gender = tree.predict(te_x_gender)

df.loc[(df['gender'].notnull()),'gender'] = tr_gender
df.loc[(df['gender'].isnull()),'gender'] = pred_gender

In [11]:
tr_major_discipline = df[df['major_discipline'].notnull()].fillna(0).astype(str).apply(LabelEncoder().fit_transform)
te_major_discipline =  df[df['major_discipline'].isnull()].fillna(0).astype(str).apply(LabelEncoder().fit_transform)

tr_y_major_discipline = tr_major_discipline['major_discipline']
tr_x_major_discipline = tr_major_discipline.drop(['major_discipline'],axis=1)

te_x_major_discipline = te_major_discipline.drop( ['major_discipline'],axis=1)

tree.fit(tr_x_major_discipline,tr_y_major_discipline)

pred_m = tree.predict(te_x_major_discipline)
df.loc[(df['major_discipline'].isnull()),'major_discipline'] = pred_m

In [12]:
tr_company = df[df['company_size'].notnull()].fillna(0).astype(str).apply(LabelEncoder().fit_transform)
te_company =  df[df['company_size'].isnull()].fillna(0).astype(str).apply(LabelEncoder().fit_transform)

tr_y_company = tr_company['company_size']
tr_x_company = tr_company.drop(['company_size'],axis=1)

te_x_company = te_company.drop( ['company_size'],axis=1)

tree.fit(tr_x_company,tr_y_company)

pred_company = tree.predict(te_x_company)

df.loc[(df['company_size'].notnull()),'company_size'] = tr_company
df.loc[(df['company_size'].isnull()),'company_size'] = pred_company

In [13]:
tr_type = df[df['company_type'].notnull()].fillna(0).astype(str).apply(LabelEncoder().fit_transform)
te_type =  df[df['company_type'].isnull()].fillna(0).astype(str).apply(LabelEncoder().fit_transform)

tr_y_type = tr_type['company_type']
tr_x_type = tr_type.drop(['company_type'],axis=1)

te_x_type = te_type.drop( ['company_type'],axis=1)

tree.fit(tr_x_type,tr_y_type)

pred_type = tree.predict(te_x_type)
df.loc[(df['company_type'].notnull()),'company_type'] = tr_type
df.loc[(df['company_type'].isnull()),'company_type'] = pred_type

In [14]:
tr_edu = df[df['education_level'].notnull()].fillna(0).astype(str).apply(LabelEncoder().fit_transform)
te_edu =  df[df['education_level'].isnull()].fillna(0).astype(str).apply(LabelEncoder().fit_transform)

tr_y_edu = tr_edu['education_level']
tr_x_edu = tr_edu.drop(['education_level'],axis=1)

te_x_edu = te_edu.drop( ['education_level'],axis=1)

tree.fit(tr_x_edu,tr_y_edu)

pred_edu = tree.predict(te_x_edu)

df.loc[(df['education_level'].notnull()),'education_level'] = tr_edu
df.loc[(df['education_level'].isnull()),'education_level'] = pred_edu

In [15]:
tr_university = df[df['enrolled_university'].notnull()].fillna(0).astype(str).apply(LabelEncoder().fit_transform)
te_university =  df[df['enrolled_university'].isnull()].fillna(0).astype(str).apply(LabelEncoder().fit_transform)

tr_y_university = tr_university['enrolled_university']
tr_x_university = tr_university.drop(['enrolled_university'],axis=1)

te_x_university = te_university.drop( ['enrolled_university'],axis=1)

tree.fit(tr_x_university,tr_y_university)

pred_university = tree.predict(te_x_university)
df.loc[(df['enrolled_university'].notnull()),'enrolled_university'] = tr_university
df.loc[(df['enrolled_university'].isnull()),'enrolled_university'] = pred_university

In [16]:
nan(df)#利用隨機森林預測遺失值、並填補之後，可以看到資料就沒有遺失值了

,total NAN,%
city,0,NaN
city_development_index,0,NaN
gender,0,NaN
relevent_experience,0,NaN
enrolled_university,0,NaN
education_level,0,NaN
major_discipline,0,NaN
experience,0,NaN
company_size,0,NaN
company_type,0,NaN


In [17]:
#將資料中，尚未轉成數值的部份轉成數值
df['relevent_experience']=pd.DataFrame(df['relevent_experience']).astype(str).apply(LabelEncoder().fit_transform)
df['major_discipline']=pd.DataFrame(df['major_discipline']).astype(str).apply(LabelEncoder().fit_transform)
df['last_new_job']=pd.DataFrame(df['last_new_job']).astype(str).apply(LabelEncoder().fit_transform)


In [18]:
#完成整理後的資料
df_new = df.astype(float)

In [19]:
#遺失值處理好了，也將字串都轉換成數值，接下來進行建模

y is target:
target is the variable we are trying to predict and calculate the probablities for.
0 – Not looking for job change, 1 – Looking for a job change.


In [20]:
x=df_new.drop(['target'],axis=1)
y=df_new['target']

In [21]:
#利用隨機森林模型，繪製各項特徵的重要性，發現每項特整都佔有一定程度的重要性，所以不刪除任何欄位
importances = tree.feature_importances_
feat_labels = df.columns[1:]
indices = np.argsort(importances)[::-1]
for f in range(x.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) training_hours                 0.225319
 2) experience                     0.144757
 3) gender                         0.105073
 4) company_size                   0.096645
 5) city_development_index         0.096049
 6) enrolled_university            0.094938
 7) last_new_job                   0.077155
 8) company_type                   0.042621
 9) education_level                0.039863
10) major_discipline               0.036644
11) target                         0.024290
12) relevent_experience            0.016644


In [22]:
#使用XGB去預測y
from xgboost import XGBClassifier
xgb = XGBClassifier()

#Fit the model
xgb.fit(x, y)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [23]:
#計算此模型的正確率
from sklearn.metrics import roc_auc_score
y_hat = xgb.predict(x)
score = roc_auc_score(y, y_hat)
print("Overall AUC: {:.3f}" .format(score))

Overall AUC: 0.779


kaggle提供test,train兩份資料集，我將train資料及整理完之後進行建模，並利用kaggle提供的test資料集，丟進模型去預測最終結果，並製作成dataframe格式，得到xgb_pred_df，並製作成kaggle要求的格式，命名為sub

In [24]:
import pickle
test = pd.read_pickle('df_new_test.pkl')
xgb_pred = xgb.predict(test)
xgb_pred_df = pd.DataFrame(xgb_pred)
xgb_pred_df

,0
0,0.0
1,0.0
2,1.0
3,0.0
4,0.0
...,...
2124,0.0
2125,0.0
2126,0.0
2127,0.0


In [25]:
#最終預測結果儲存成csv
sub = pd.read_csv('/Users/Ruby/Desktop/sample_submission.csv')
sub['target'] = xgb_pred_df

In [26]:
#sub.to_pickle('sub.csv')
#資料利用pickle儲存

kaggle有提供答案，將預測結果sub與答案answer進行比對，計算出正確率

In [27]:
#下載正確解答
answer = np.load('/Users/Ruby/Downloads/jobchange_test_target_values.npy') 
answer_df = pd.DataFrame(answer)

In [28]:
#將正確解答與我們的預測值進行比較，計算正確率
y_hat = xgb.predict(test)
score = roc_auc_score(answer_df, y_hat)
print("Overall AUC: {:.3f}" .format(score))

Overall AUC: 0.618
